In [4]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [5]:
from src.inference import get_feature_store

In [6]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='US/Eastern')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from ),
    end_time=(fetch_data_to),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-05 20:34:18,624 INFO: Initializing external client
2025-03-05 20:34:18,630 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 20:34:19,700 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214717
Fetching data from 2025-02-04 20:34:18.618235-05:00 to 2025-03-05 19:34:18.618235-05:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.29s) 


In [7]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-05 02:00:00+00:00,2,0
1,2025-02-05 03:00:00+00:00,2,0
2,2025-02-05 04:00:00+00:00,2,0
3,2025-02-05 05:00:00+00:00,2,0
4,2025-02-05 06:00:00+00:00,2,0
...,...,...,...
173623,2025-03-05 14:00:00+00:00,263,136
173624,2025-03-05 15:00:00+00:00,263,153
173625,2025-03-05 16:00:00+00:00,263,181
173626,2025-03-05 17:00:00+00:00,263,215


In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173628 entries, 0 to 173627
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         173628 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  173628 non-null  int32                  
 2   rides               173628 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 2.6 MB


In [9]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [10]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173628 entries, 0 to 173627
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         173628 non-null  datetime64[us]
 1   pickup_location_id  173628 non-null  int32         
 2   rides               173628 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 2.6 MB


In [11]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [12]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,32,48,145,8,155,145,62,60,6,80,...,72,31,34,108,123,55,132,56,263,2025-02-11 04:00:00
1,0,0,0,0,0,0,0,0,0,1,...,2,0,0,0,0,1,1,1,210,2025-02-28 04:00:00
2,3,0,0,0,0,0,1,0,2,0,...,0,0,3,2,0,3,0,1,216,2025-03-03 20:00:00
3,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,2,0,217,2025-02-26 08:00:00
4,0,8,1,0,6,10,9,3,7,6,...,37,41,4,22,48,6,14,2,261,2025-02-12 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,47,2025-02-24 19:00:00
248,2,2,1,1,3,0,0,0,1,1,...,2,13,5,6,1,0,2,1,97,2025-02-23 10:00:00
249,0,2,1,1,0,0,0,5,0,1,...,0,1,1,2,1,0,0,2,82,2025-02-09 02:00:00
250,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,73,2025-02-22 00:00:00


In [20]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='US/Eastern')
features = load_batch_of_features_from_store(current_date)

2025-03-05 20:35:22,204 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 20:35:22,210 INFO: Initializing external client
2025-03-05 20:35:22,210 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 20:35:22,857 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214717
Fetching data from 2025-02-04 20:35:22.204301-05:00 to 2025-03-05 19:35:22.204301-05:00
2025-03-05 20:36:19,841 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 209, in __init__
    self._health_check()
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\

2025-03-05 20:36:26,104 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 394, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 459, in read_query
    return self._get_dataset(
           ^^^^^^^^^^^^^^^^^^
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 257, in call
    return attempt.get(self._wrap_exception)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\retrying.py", line 301, i

AttributeError: 'ArrowFlightClient' object has no attribute '_server_version'

In [14]:
current_date

Timestamp('2025-03-06 01:34:30.908364+0000', tz='Etc/UTC')

In [15]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-05 02:00:00
1,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,3,2025-03-05 02:00:00
2,3,0,0,0,0,4,6,2,3,2,...,5,3,3,5,3,2,5,3,4,2025-03-05 02:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-05 02:00:00
4,0,0,0,1,2,6,4,2,1,4,...,1,2,0,2,0,1,1,0,7,2025-03-05 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,0,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-05 02:00:00
248,0,0,0,0,0,1,4,1,0,3,...,1,1,0,1,0,1,1,0,260,2025-03-05 02:00:00
249,0,1,0,1,7,8,15,14,12,23,...,46,34,25,23,22,13,4,1,261,2025-03-05 02:00:00
250,0,5,6,13,66,151,147,158,114,84,...,87,48,30,25,10,5,1,2,262,2025-03-05 02:00:00


In [16]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 20:34:46,481 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 20:34:46,494 INFO: Initializing external client
2025-03-05 20:34:46,495 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 20:34:47,176 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214717


In [17]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1538, in _execute_child
   

In [18]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,1.0
3,6,0.0
4,7,0.0
...,...,...
247,259,0.0
248,260,0.0
249,261,1.0
250,262,1.0


In [19]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([ 48,  79, 249, 230, 132, 186, 246, 100, 114, 164])